# Part 2: Batch Process

In the following notebook you will use the parameters found in A_Evaluation&Testing.ipynb to correct and stitch entire datasets.

## 1. Import packages. 
*This must be done every time the notebook is started or restarted.

In [1]:
from concurrent.futures import ThreadPoolExecutor
import gc
import os as os
from tqdm.notebook import tqdm
import pandas as pd
import KCorrect
from Kstitch.stitching import stitch_images
from glob import glob
from skimage.io.collection import alphanumeric_key
import numpy as np
from skimage.io import imread_collection, imsave
import stackview
import numpy as np
from itertools import chain, repeat
import subprocess
from datetime import datetime
import imagej, scyjava
import logging
import pyopencl as cl
import warnings
import platform
os_system = platform.system()
current_dateTime = datetime.now()

## 2. Define directory paths. 
*This must be done every time the notebook is started or restarted.

In [2]:
base_dir = "C:\\Users\\smith6jt"

In [3]:
image_dir = os.path.join(base_dir, 'KINTSUGI', 'data', '1904_CC2B_raw')
stitch_dir = image_dir.replace("_raw", "_BaSiC_Stitched")
meta_dir = stitch_dir.replace("_BaSiC_Stitched", "_meta")
project_file = os.path.join(meta_dir, "project_data.txt")
print(f"Image folder is {image_dir}.")
print(f"Stitching folder is {stitch_dir}.")
print(f"Meta folder is {meta_dir}.")

Image folder is C:\Users\smith6jt\KINTSUGI\data\1904_CC2B_raw.
Stitching folder is C:\Users\smith6jt\KINTSUGI\data\1904_CC2B_BaSiC_Stitched.
Meta folder is C:\Users\smith6jt\KINTSUGI\data\1904_CC2B_meta.


## 3. Stitching and Illumination Correction

### 3.1 Stitching Function

- The following cell defines the function that is called from the apply_basic function to stitch the corrected tiles.  
- It creates a file with the stitching positions calculated from the middle z-plane images of the first channel.
- The file is then used to stitch the rest of the images for all channels in the cycle. BECAUSE OF THIS, YOU MUST HAVE RUN THE FIRST CHANNEL OF A CYCLE TO STITCH THE OTHER CHANNELS.

In [7]:
def stitch(images_transformed, zplanes, dest, dest_1, channels, zplanes_n, pou, rows, cols, overlap_percentage, use_gpu):
    
    z = str(zplanes)
    ch = str(channels)
    pkl_dest = os.path.join(dest, "result_df.pkl")
    
    if zplanes == zplanes_n//2 and channels == 1:
        print(f"Start Stitching Z0{z.zfill(2)}_CH{ch}")
        if os.path.exists(pkl_dest):
            result_df = pd.read_pickle(pkl_dest)
        else:
            result_df, _ = stitch_images(images_transformed, rows, cols, initial_ncc_threshold=0.0, overlap_percentage=overlap_percentage, pou=pou, use_gpu=use_gpu)
            result_df.to_pickle(pkl_dest)
           
        
    else:
        print(f"Start Stitching Z0{z.zfill(2)}_CH{ch}")
        if os.path.exists(os.path.join(dest_1, "result_df.pkl")):
            result_df = pd.read_pickle(os.path.join(dest_1, "result_df.pkl"))

        else:
            print("Run registration channel to produce a stitching model.")

    result_df["y_pos2"] = result_df["y_pos"] - result_df["y_pos"].min()
    result_df["x_pos2"] = result_df["x_pos"] - result_df["x_pos"].min()
    
    size_y = images_transformed.shape[1]
    size_x = images_transformed.shape[2]
    
    stitched_image_size = (
        result_df["y_pos2"].max() + size_y,
        result_df["x_pos2"].max() + size_x,
    )
    stitched_image = np.zeros_like(images_transformed, shape=stitched_image_size)
    
    for i, row in result_df.iterrows():
        stitched_image[
            row["y_pos2"] : row["y_pos2"] + size_y,
            row["x_pos2"] : row["x_pos2"] + size_x,
        ] = images_transformed[i]

    result_image_file_path = os.path.join(dest,f"{z.zfill(2)}.tif") 
    imsave(result_image_file_path, stitched_image, check_contrast=False)
    
    print(f"Saved to {result_image_file_path}")


### 3.2 Illumination Correction Function

- The following cell contains the function that will apply the illumination correction to your data.  
- Change the five variables below according to tests using the first notebook.

In [8]:
def apply_KCorrect(image_dir, stitch_dir, zplanes, cycles, channels, zplanes_n, pou, rows, cols, overlap_percentage, use_gpu):

    if_darkfield = True
    max_iterations = 500
    optimization_tolerance = 1e-6
    max_reweight_iterations = 25
    reweight_tolerance = 1.0e-3

    filename_pattern = f'1_000??_Z0{str(zplanes).zfill(2)}_CH{str(channels)}.tif'
    
    dest = os.path.join(stitch_dir, f"cyc{str(cycles).zfill(2)}", f"CH{str(channels)}")
    os.makedirs(dest, exist_ok=True)
    dest_1 = os.path.join(stitch_dir, f"cyc{str(cycles).zfill(2)}", "CH1")
    
    im_raw = sorted(glob(os.path.join(image_dir, f'cyc{str(cycles).zfill(3)}', filename_pattern)), key=alphanumeric_key)
    im = imread_collection(im_raw)
    im_array_init = np.asarray(im)
    dtype_max = np.iinfo(im_array_init.dtype).max
    im_array = im_array_init.astype(np.float64) / dtype_max

    if not np.all(np.isfinite(im_array)):
        raise ValueError("Input array contains inf or nan values")
    if np.any(im_array < 0):
        raise ValueError("Input array contains negative values")

    print(f"Start Illumination Correction cyc{str(cycles).zfill(2)} Z0{str(zplanes).zfill(2)}_CH{str(channels)}")
    flatfield, darkfield = KCorrect.KCorrect(im_array, if_darkfield = if_darkfield, max_iterations = max_iterations, optimization_tolerance = optimization_tolerance,  max_reweight_iterations = max_reweight_iterations, reweight_tolerance = reweight_tolerance)
    
    if np.any(np.isnan(flatfield)) or np.any(np.isnan(darkfield)):
        raise ValueError("Invalid flatfield or darkfield correction")
    if np.any(flatfield == 0):
        warnings.warn("Flatfield contains zero values which may cause division issues")
    
    corrected = np.zeros_like(im_array, dtype=np.float64)
    for i in range(len(im_array)):
        corrected[i] = ((im_array[i] - darkfield) / flatfield)
        corrected[i] = np.clip(corrected[i], 0, 1)
        
    KCorrect.validate_correction(im_array_init, corrected)
    corrected = (corrected * dtype_max).astype(np.uint16)                                       
    stitch(corrected, zplanes, dest, dest_1, channels, zplanes_n, pou, rows, cols, overlap_percentage, use_gpu)


### 3.3 Running Multiple Cycle/Channel Combinations

- This cell runs the above two functions for all the images you define with start_cycle, end_cycle, start_channel, and end_channel.  
- Enter values for n, m, pou (determined from the first notebook), zplanes_n (number of zplanes), and overlap_percentage.  
- Enter the number of workers for the multithreading (usually determined by dataset size, available memory, and number of cores).  See https://docs.python.org/3/library/concurrent.futures.html
- To run just one cycle or channel, simply enter the same number for both start and end.

In [ ]:
n = 9  # Number of rows (height)
m = 7  # Number of columns (width)
start_cycle = 1
end_cycle = 13
start_channel = 1
end_channel = 4
pou = 13
zplanes_n = 17
overlap_percentage = 0.30
workers = zplanes_n-1
use_gpu = True

# Row coordinates: each row index is repeated m times
rows = list(chain.from_iterable(repeat(row, m) for row in range(n)))
# Column coordinates: snake pattern for each row, going back and forth from top left going right
cols = list(chain.from_iterable(range(m) if row % 2 == 0 else range(m - 1, -1, -1) for row in range(n)))

image_dir_list = [image_dir] * (zplanes_n-1)
stitch_dir_list = [stitch_dir] * (zplanes_n-1)
zplanes_list =[zplanes_n] * (zplanes_n-1)
pou_list = [pou] * (zplanes_n-1)
rows_list = [rows] * (zplanes_n-1)
cols_list = [cols] * (zplanes_n-1)
overlap_percentage_list = [overlap_percentage] * (zplanes_n-1)
use_gpu_list = [use_gpu] * (zplanes_n-1)


total_operations = (end_cycle - start_cycle + 1) * (end_channel - start_channel + 1)

with tqdm(total=total_operations, desc='Processing', unit='operation',
                    bar_format='{desc}: {percentage:3.0f}%|{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}]',
                    colour="green", position=0, leave=True) as pbar:
    for i in range(start_cycle, end_cycle+1):
        for j in range(start_channel, end_channel+1):
            pbar.set_description(f'Cycle {i} Channel {j}')
            cycles = i
            zplanes = zplanes_n//2 
            channels = display_jpeg
            apply_KCorrect(image_dir, stitch_dir, zplanes, cycles, channels, zplanes_n, pou, rows, cols, overlap_percentage, use_gpu)
            
            if __name__ ==  '__main__':
                with ThreadPoolExecutor(max_workers=workers) as executor:
                    cycles = [i] * (zplanes_n-1) 
                    zplanes =  list(range(1, zplanes_n//2))+list(range((zplanes_n//2)+1, zplanes_n+1)) 
                    channels = [j] * (zplanes_n-1)
                    executor.map(apply_KCorrect, image_dir_list, stitch_dir_list, zplanes, cycles, channels, zplanes_list, pou_list, rows_list, cols_list, overlap_percentage_list, use_gpu_list) 

            pbar.update(1)
        gc.collect()

pbar.close()

## 4. Deconvolution

- See the first notebook for information.

In [12]:
def run_subprocess(command):
    
    process = subprocess.Popen(command, 
                                stdout=subprocess.PIPE, 
                                stderr=subprocess.STDOUT, 
                                shell=True, 
                                bufsize=1,
                                universal_newlines=True, 
                                text=True)
    
    while True:
        output = process.stdout.readline()
        if output == '' and process.poll() is not None:
            break
        if output:
            print(output, end='') 

    rc = process.poll()
    return rc

In [13]:
def decon(base_dir, stitch_dir, dec_cycle, dec_channel):

    # pixel size in xy dimension (nanometers)
    xy_vox = 377
    # pixel size in z dimension (nanometers)
    z_vox = 1500
    # Number of iterations of Lucy-Richardson algo before stopping unless stop_crit is met first
    iterations = 25
    # Microscope objective numerical aperture
    mic_NA = 0.75
    # Refractive index of tissue being imaged
    tissue_RI = 1.44
    # Opening size in millimeters of objective aperture
    slit_aper = 6.5
    # Focal length in millimeters of objective
    f_cyl = 1
    # Used to reduce noise.  Increase value for noisy images. (0-10)
    damping = 0
    # If set, the deconvolved images will be clipped by this percent for max and min values, and then scaled to full range of bit depth. (0-5)
    hist_clip = 0.01
    # Percent change between iterations to use as criteria to stop deconvolution.
    stop_criterion = 5.00
    # Percent maximum GPU or CPU memory 
    max_memory = 0.7
    # Enter 'GPU' or 'CPU'
    device = 'GPU'
    # Chunk size in bytes, or 1 for automatic chunking
    blocksize = 3072
    # The respective excitation and emission wavelength in nanometers for each channel
    wavelengths = {
        1: (358, 461),
        2: (753, 775),
        3: (560, 575),
        4: (648, 668)
    }
    
    decon_dir = stitch_dir.replace('_BaSiC_Stitched', '_Decon')
    source = os.path.join(stitch_dir, f"cyc{str(dec_cycle).zfill(2)}", f"CH{dec_channel}")
    dest = os.path.join(decon_dir, f"cyc{str(dec_cycle).zfill(2)}", f"CH{dec_channel}")
    os.makedirs(dest, exist_ok=True)

    command = [source, str(xy_vox), str(z_vox), str(iterations), str(mic_NA), str(tissue_RI), str(f_cyl), str(slit_aper), str(damping), str(hist_clip), str(stop_criterion), str(max_memory), device, str(blocksize)]

    if os_system == "Windows":

        decon_exe = os.path.join(base_dir, "KINTSUGI", "KDecon_v4.exe") 
        command.insert(0, decon_exe)
        ex_index, em_index = 7, 8

    if os_system == "Linux":

        mat_dir = "/usr/local/MATLAB/MATLAB_Runtime/R2024a/" 
        decon_exe = os.path.join(base_dir, "KINTSUGI", "for_distribution_files_only", "run_K_Decon_v4.sh")
        command.insert(0, decon_exe)
        command.insert(1, mat_dir)
        ex_index, em_index = 8, 9

    ex, em = wavelengths[dec_channel]
    command.insert(ex_index, str(ex))
    command.insert(em_index, str(em))

    run_subprocess(command)
    os.rename(os.path.join(source, 'deconvolved'), os.path.join(dest, 'deconvolved'))

To apply the above decon function to multiple cycles/channels, enter the start and end numbers below.

In [ ]:
decon_start_cycle = 1
decon_end_cycle = 13

decon_start_channel = 1
decon_end_channel = 4

total_operations = (decon_end_cycle - decon_start_cycle + 1) * (decon_end_channel - decon_start_channel + 1)

with tqdm(total=total_operations, desc='Processing', unit='operation',
                    bar_format='{desc}: {percentage:3.0f}%|{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}]',
                    colour="green", position=0, leave=True) as pbar:
    for i in range(decon_start_cycle, decon_end_cycle+1):
        for j in range(decon_start_channel, decon_end_channel+1):
            pbar.set_description(f'Cycle {i} Channel {j}')
            decon(base_dir, stitch_dir, i, j)
            pbar.update(1)
        gc.collect()

pbar.close()

## 5. FIJI Clij2 plugin - EDF

See the previous notebook for information.

In [4]:
fiji_dir = os.path.join(base_dir, "Fiji.app")
javahome = os.path.join(base_dir, "KINTSUGI", "jdk-21_windows-x64_bin", "jdk-21.0.5", "bin")
os.environ['PATH'] = javahome + os.pathsep + os.environ['PATH']
mavenhome = os.path.join(base_dir, 'KINTSUGI','apache-maven-3.9.9-bin','apache-maven-3.9.9', 'bin')
os.environ['PATH'] = mavenhome + os.pathsep + os.environ['PATH']

ij_mem = 50
scyjava.config.add_option(f'-Xmx{str(ij_mem)}g')
ij = imagej.init(fiji_dir)

In [ ]:
platforms = cl.get_platforms()
for platform in platforms:
    print(f"Platform: {platform.name}")
    devices = platform.get_devices()
    for device in devices:
        print(f"  Device: {device.name} - Type: {cl.device_type.to_string(device.type)}")

cpu_device = None
for platform in platforms:
    devices = platform.get_devices()
    for device in devices:
        if device.type == cl.device_type.CPU:
            cpu_device = device
            break
    if cpu_device:
        break

In [5]:
channel_name_dict = {"cyc01.tif" : ["DAPI", "Blank1a", "Blank1b", "Blank1c"],
 "cyc02.tif" : ["DAPI", "CD31", "CD8", "Empty2c"],
 "cyc03.tif" : ["DAPI", "CD20", "Ki67", "CD3e"],
 "cyc04.tif" : ["DAPI", "SMActin", "Podoplanin", "CD68"],
 "cyc05.tif" : ["DAPI", "PanCK", "CD21", "CD4"],
 "cyc06.tif" : ["DAPI", "Lyve1", "CD45RO", "CD11c"],
 "cyc07.tif" : ["DAPI", "CD35", "ECAD", "CD107a"],
 "cyc08.tif" : ["DAPI", "CD34", "CD44", "HLADR"],
 "cyc09.tif" : ["DAPI", "Empty9a", "FoxP3", "CD163"],
 "cyc10.tif" : ["DAPI", "Empty10a", "CollagenIV", "Vimentin"],
 "cyc11.tif" : ["DAPI", "Empty11a", "CD15", "CD45"],
 "cyc12.tif" : ["DAPI", "Empty12a", "CD5", "CD1c"],
 "cyc13.tif" : ["DAPI", "Blank13a", "Blank13b", "Blank13c"]}

In [6]:
edf_start_cycle = 1
edf_end_cycle = 13

edf_start_channel = 1
edf_end_channel = 4

decon_dir = stitch_dir.replace('_BaSiC_Stitched', '_Decon')
edf_dir = decon_dir.replace('_Decon', '_EDF')

In [7]:
device = "NVIDIA RTX A4500"

radius_x = 5
radius_y = 5
sigma = 20.0

xSplit = 3
ySplit = 3

z_start = 3
z_end = 15

GPU = True

In [8]:
macro = """
#@ File in_folder
#@ String device
#@ File out_folder
#@ String file_name
#@ Integer radius_x
#@ Integer radius_y
#@ Float sigma
#@ Integer xTiles
#@ Integer yTiles
#@ String z_start
#@ String z_end
#@ Boolean GPU

run("CLIJ2 Macro Extensions", "cl_device=[" + device + "] automatic_output_naming=false");
Ext.CLIJ2_clear();
//setBatchMode(true);

File.openSequence(in_folder);
Stack.getDimensions(width, height, channels, slices, frames);

if (slices != z_end - z_start + 1) {
    run("Slice Keeper", "first=" + z_start + " last=" + z_end + " increment=1");
    input = "deconvolved kept stack";
    Stack.getDimensions(width, height, channels, slices, frames);
    } else {
    input = "deconvolved";
}

if (GPU == true) {
	output = "output";
	newImage(output, "16-bit black", width, height, 1);
    } else {
    Ext.CLIJ2_push(input);
	Ext.CLIJ2_create2D(output, width, height, 16);
}

numTilesZ = 1;
numTilesX = xTiles;
numTilesY = yTiles;

tileDepth = round((slices/numTilesZ));
tileWidth = round((width/numTilesX));
tileHeight = round((height/numTilesY));

for (x = 0; x < numTilesX; x++) {
	for (y = 0; y < numTilesY; y++) {
		for (z = 0; z < numTilesZ; z++) {

			Ext.CLIJ2_pushTile(input, x, y, z, tileWidth, tileHeight, tileDepth, 0, 0, 0);		
			Ext.CLIJ2_extendedDepthOfFocusVarianceProjection(input, output, radius_x, radius_y, sigma);
            Ext.CLIJ2_release(input);
            if (GPU == true) {
				Ext.CLIJ2_pullTile(output, x, y, z, tileWidth, tileHeight, 1, 0, 0, 0);
				Ext.CLIJ2_release(output);
			}            
		}
	}
}

if (GPU == false) {
	Ext.CLIJ2_pull(output);
    Ext.CLIJ2_release(output);
}

selectImage(output);
saveAs("Tiff", out_folder + File.separator + file_name);

Ext.CLIJ2_clear();
run("Close All");
"""

In [9]:
total_operations = (edf_end_cycle - edf_start_cycle + 1) * (edf_end_channel - edf_start_channel + 1)

with tqdm(total=total_operations, desc='Processing', unit='operation',
                    bar_format='{desc}: {percentage:3.0f}%|{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}]',
                    colour="green", position=0, leave=True) as pbar:
    for i in range(edf_start_cycle, edf_end_cycle+1):
        edf_dest = os.path.join(edf_dir, f"cyc{str(i).zfill(2)}")
        os.makedirs(edf_dest, exist_ok=True)
        
        for j in range(edf_start_channel, edf_end_channel+1):

            pbar.set_description(f'Cycle {i} Channel {j}')
            edf_source = os.path.join(decon_dir, f"cyc{str(i).zfill(2)}", f"CH{str(j)}", "deconvolved")
            file_name = channel_name_dict.get(f"cyc{str(i).zfill(2)}.tif")[j-1]
            

            args = {'in_folder': edf_source, 
                    "device" : device, 
                    "out_folder" : edf_dest, 
                    "file_name" : file_name, 
                    "radius_x" : radius_x, 
                    "radius_y" : radius_y, 
                    "sigma" : sigma, 
                    "xTiles" : xSplit, 
                    "yTiles" : ySplit, 
                    "z_start" : z_start, 
                    "z_end" : z_end, 
                    "GPU" : GPU}

            ij.py.run_macro(macro, args)

            pbar.update(1)
        gc.collect()

pbar.close()  

Processing:   0%|          | 0/52 [00:00<?]

Operating in headless mode - the original ImageJ will have limited functionality.


## 6. Registration

- Registration is performed using the VALIS program https://valis.readthedocs.io/en/stable/examples.html. 
- In the following cell, the images produced in Section 5 are combined to multi-channel tif files in a format the VALIS program is compatible with.
- Enter the cycles being processed, the data_type desired, and the pixel size.
- The metadata and parameters for pyramidal output may be changed.
- Be sure the path to the vips binaries is defined correctly.

### 6.1 Combine channels for each cycle

In [ ]:
reg_start_cycle = 1
reg_end_cycle = 13
data_type = "uint16"  
pixel_size = float(0.377)

logging.basicConfig(level=logging.WARN) #change to INFO for more details

if os_system == "Windows":
    vipshome = os.path.join(base_dir, 'KINTSUGI','vips-dev-8.16','bin')
    os.environ['PATH'] = vipshome + ';' + os.environ['PATH']

import pyvips 

edf_dir = stitch_dir.replace('_BaSiC_Stitched', '_EDF')
reg_dir = edf_dir.replace('_EDF', '_Registration')
os.makedirs(reg_dir, exist_ok=True)

pbar_filesave = tqdm(total=100, unit="Percent",
                    bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}]',
                    colour="green", position=0, leave=True)

for j in range(reg_start_cycle, reg_end_cycle + 1):

    cycle= f"cyc{str(j).zfill(2)}" 
     
    reg_source = os.path.join(edf_dir, f"cyc{str(j).zfill(2)}") 
    image_set = glob(os.path.join(reg_source, '*.tif'))
    channel_name_order = channel_name_dict.get(f"cyc{str(j).zfill(2)}.tif")
    image_set.sort(key=lambda x: channel_name_order.index(os.path.basename(x).split('.')[0]))
  
    channel_names = []
    for i in range(len(image_set)):
        split_name = os.path.basename(image_set[i].split(".")[0])
        channel_names.append(split_name)
 
    pyvips_image_set = [pyvips.Image.new_from_file(os.path.join(reg_source, filename), access="sequential") for filename in image_set]

    out_init = pyvips.Image.arrayjoin(pyvips_image_set, across=1)
    out = out_init.copy()
    out.set_type(pyvips.GValue.gint_type, "page-height", pyvips_image_set[0].height)

    x_dim = pyvips_image_set[0].width
    y_dim = pyvips_image_set[0].height

    bands = len(pyvips_image_set)
    outfile = os.path.join(reg_dir, f'cyc{str(j).zfill(2)}.ome.tif')
    out.set_type(pyvips.GValue.gstr_type, "image-description",
    f"""<?xml version="1.0" encoding="UTF-8"?>
        <OME xmlns="http://www.openmicroscopy.org/Schemas/OME/2016-06"
            xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"
            xsi:schemaLocation="http://www.openmicroscopy.org/Schemas/OME/2016-06 http://www.openmicroscopy.org/Schemas/OME/2016-06/ome.xsd">
            <Image ID="Image:0" Name="{cycle}" >
                <!-- Minimum required fields about image dimensions -->
                <Pixels BigEndian="false"
                        DimensionOrder="XYCZT" 
                        ID="Pixels:0"
                        Interleaved="false"
                        SignificantBits="16"                                              
                        SizeC="{bands}" 
                        SizeT="1" 
                        SizeX="{x_dim}" 
                        SizeY="{y_dim}" 
                        SizeZ="1" 
                        Type="{data_type}" 
                        PhysicalSizeX="{pixel_size}" 
                        PhysicalSizeY="{pixel_size}">
                    <TiffData FirstC="0" FirstT="0" FirstZ="0" IFD="0" PlaneCount="1"/>
                    <Channel Color="16751615" ID="Channel:0:0" Name="{channel_names[0]}" IlluminationType="Epifluorescence" ContrastMethod="Fluorescence" AcquisitionMode="WideField" SamplesPerPixel="1"/>
                    <TiffData FirstC="1" FirstT="0" FirstZ="0" IFD="1" PlaneCount="1"/>
                    <Channel Color="7995391" ID="Channel:0:1" Name="{channel_names[1]}" IlluminationType="Epifluorescence" ContrastMethod="Fluorescence" AcquisitionMode="WideField" SamplesPerPixel="1"/>
                    <TiffData FirstC="2" FirstT="0" FirstZ="0" IFD="2" PlaneCount="1"/>
                    <Channel Color="9043967" ID="Channel:0:2" Name="{channel_names[2]}" IlluminationType="Epifluorescence" ContrastMethod="Fluorescence" AcquisitionMode="WideField" SamplesPerPixel="1"/>
                    <TiffData FirstC="3" FirstT="0" FirstZ="0" IFD="3" PlaneCount="1"/>
                    <Channel Color="1828651263" ID="Channel:0:3" Name="{channel_names[3]}" IlluminationType="Epifluorescence" ContrastMethod="Fluorescence" AcquisitionMode="WideField" SamplesPerPixel="1"/>
                </Pixels>
            </Image>
        </OME>""")

    out.tiffsave(outfile, subifd=True, page_height=y_dim, compression='lzw', tile=True,
                  tile_width=512, tile_height=512, pyramid=True, bigtiff=True)
    out_init = None
    out = None
    pyvips_image_set = None
    gc.collect() 
    pbar_filesave.update(100 / (reg_end_cycle - reg_start_cycle + 1))

pbar_filesave.close()

### 6.2 VALIS Registration

- See https://github.com/MathOnco/valis/tree/main for full documentation. 
The process for registration involves producing registration data on the rigid and non-rigid movement of the images relative to one another to bring about alignment as they overlap.  This data is inspected for accuracy as the writing of the registered images takes a long time.  There is an optional micro-registration if smaller structure overlap is important.  Either way, the registration results to be used for the image registration is stored in the results_dst_dir as a .pickle file.
- According to the VALIS docs: One of the most important parameters used to initialize a Valis object is max_processed_image_dim_px, which determines the size of the image used to find the rigid registration parameters. The default value is 850, but if registration fails or is poor, try adjusting that value. Generally speaking, values between 500-2000 work well. In cases where there is little empty space around the tissue, smaller values may be better. However, if there is a large amount of empty space/slide (as in the images above), larger values may be needed so that the tissue is at a high enough resolution. To improve alignment of the finer details in the images, larger images can be used in the non-rigid or micro-registration steps (set via the max_non_rigid_registration_dim_px parameter).
- If you get an error that the jvm is not found, try uncommenting the registration.init_jvm() line.


In [4]:
os_system = platform.system()
if os_system == "Windows":
    vipsbin = os.path.join(base_dir, 'KINTSUGI','vips-dev-8.16','bin')
    os.environ['PATH'] = vipsbin + ';' + os.environ['PATH']
javahome = os.path.join(base_dir, "KINTSUGI", "jdk-21_windows-x64_bin", "jdk-21.0.5", "bin")
os.environ['PATH'] = javahome + os.pathsep + os.environ['PATH']

mavenhome = os.path.join(base_dir, 'KINTSUGI','apache-maven-3.9.9-bin','apache-maven-3.9.9', 'bin')
os.environ['PATH'] = mavenhome + os.pathsep + os.environ['PATH']

from valis import registration
warnings.filterwarnings("ignore")

# registration.init_jvm()

edf_dir = stitch_dir.replace('_BaSiC_Stitched', '_EDF')
reg_dir = edf_dir.replace('_EDF', '_Registration')

slide_src_dir = reg_dir
results_dst_dir = os.path.join(reg_dir, "valis_out")


- Running this cell will produce registration data, but will not register the images, allowing for testing different parameters.

- Note that if you rerun the registration, results will be overwritten.  To avoid this you will need to rename your results_dst_dir folder.


In [ ]:
max_processed_image_dim_px = 600
registrar = registration.Valis(slide_src_dir, results_dst_dir, crop="overlap", max_processed_image_dim_px = max_processed_image_dim_px, imgs_ordered=True)
rigid_registrar, non_rigid_registrar, error_df = registrar.register()
registration.kill_jvm()

- Review the results in the produced registration folder saved in “valis_out”
- A quick method is to scroll through the “deformation_fields” images below.  Be cautious of angular deformations.  Other options to view by replacing "deformation_fields" below: "masks", "non_rigid_registration", "processed", and "rigid_registration".
- To view the output, you may need to click the ellipses next to the output cell and click "Change Presentation" to choose IPyWidgetRenderer.

In [ ]:
# Other options to view by replacing "deformation_fields" below: "masks", "non_rigid_registration", "processed", and "rigid_registration".
im_raw_deform = glob(os.path.join(reg_dir, "valis_out", os.path.basename(reg_dir), "deformation_fields", "*.png"))
im_deform = imread_collection(im_raw_deform)
deform = np.asarray(im_deform).astype(np.uint8)

stackview.slice(deform, zoom_factor=1.6, continuous_update=True)

- Micro-registration takes much longer (e.g. standard ~5-10 min and micro ~2-3 hrs) than standard registration.  Be sure to test a few changes to the standard method before switching to micro.
- If further refining of results is warranted, the optional micro-registration may be run.
- Note that the previous registration is saved to the .pickle file in path_to_register.  This will be overwritten by the micro-registration results.  To avoid this, rename the .pickle file.
- The cell below determines the parameter to use based on 25% of original resolution.

In [ ]:
micro_reg_fraction = 0.25
img_dims = np.array([slide_obj.slide_dimensions_wh[0] for slide_obj in registrar.slide_dict.values()])
print(img_dims)
min_max_size = np.min([np.max(d) for d in img_dims])
img_areas = [np.multiply(*d) for d in img_dims]
max_img_w, max_img_h = tuple(img_dims[np.argmax(img_areas)])
micro_reg_size = np.floor(min_max_size*micro_reg_fraction).astype(int)

In [ ]:
from valis.micro_rigid_registrar import MicroRigidRegistrar

micro_results_dst_dir = os.path.join(reg_dir, "valis_out_micro")
registrar = registration.Valis(slide_src_dir, micro_results_dst_dir, micro_rigid_registrar_cls=MicroRigidRegistrar)
rigid_registrar, non_rigid_registrar, error_df = registrar.register()
micro_reg, micro_error = registrar.register_micro(max_non_rigid_registration_dim_px=micro_reg_size)
matches_dst_dir = os.path.join(registrar.dst_dir, "hi_rez_matches")
registrar.draw_matches(matches_dst_dir)
registration.kill_jvm()

- Review the results of micro-registration.
- New output will be for "deformation_fields", "micro_registration", "non_rigid_registration", "overlaps", and "hi_rez_matches".

In [ ]:
im_raw_micro = glob(os.path.join(reg_dir, "valis_out", os.path.basename(reg_dir), "deformation_fields", "*.png"))
im_micro = imread_collection(im_raw_micro)
micro = np.asarray(im_micro).astype(np.uint8)

stackview.slice(micro, zoom_factor=1.6, continuous_update=True)

Here is where the application of the registration to the images occurs.  The first cell merges all images to one file, while the second creates one image per cycle. If multiple rounds of testing was done, be sure you are using the final .pickle file for the final image creation.
- At this point, images are likely not the same size.  Using the crop function will crop all images to the same size.  This is necessary to use image math functions in the next notebook.
- Naming is based on the definitions of “channel_name_dict” defined above.  It may be necessary to run that cell again.
- The registration of the common first channel will effectively align the rest of the channels in each cycle to one another.
- Note that writting large image files generally takes a long time.

In [6]:
channel_name_dict = {"cyc01.tif" : ["DAPI", "Blank1a", "Blank1b", "Blank1c"],
 "cyc02.tif" : ["DAPI", "CD31", "CD8", "Empty2c"],
 "cyc03.tif" : ["DAPI", "CD20", "Ki67", "CD3e"],
 "cyc04.tif" : ["DAPI", "SMActin", "Podoplanin", "CD68"],
 "cyc05.tif" : ["DAPI", "PanCK", "CD21", "CD4"],
 "cyc06.tif" : ["DAPI", "Lyve1", "CD45RO", "CD11c"],
 "cyc07.tif" : ["DAPI", "CD35", "ECAD", "CD107a"],
 "cyc08.tif" : ["DAPI", "CD34", "CD44", "HLADR"],
 "cyc09.tif" : ["DAPI", "Empty9a", "FoxP3", "CD163"],
 "cyc10.tif" : ["DAPI", "Empty10a", "CollagenIV", "Vimentin"],
 "cyc11.tif" : ["DAPI", "Empty11a", "CD15", "CD45"],
 "cyc12.tif" : ["DAPI", "Empty12a", "CD5", "CD1c"],
 "cyc13.tif" : ["DAPI", "Blank13a", "Blank13b", "Blank13c"]}

In [ ]:
merged_slide_dst_f = os.path.join(results_dst_dir, "merged.ome.tif")
path_to_registrar = os.path.join(reg_dir, "valis_out", os.path.basename(reg_dir),"data", os.path.basename(reg_dir)+"_registrar.pickle")
registrar = registration.load_registrar(path_to_registrar)

merged_img, channel_names, ome_xml = registrar.warp_and_merge_slide(merged_slide_dst_f,
                                    crop="reference",
                                    drop_duplicates=True, 
                                    compression="jp2k",
                                    Q=90,
                                    pyramid=False)

registration.kill_jvm()

In [ ]:
path_to_registrar = os.path.join(reg_dir, "valis_out", os.path.basename(reg_dir),"data", os.path.basename(reg_dir)+"_registrar.pickle")
registrar = registration.load_registrar(path_to_registrar)

for slide_name, slide_obj in registrar.slide_dict.items():
    reg_dir_out = os.path.join(reg_dir, "registered")
    os.makedirs(reg_dir_out, exist_ok=True)
    dst_f = os.path.join(reg_dir_out, f"{slide_name}.ome.tif")
    slide_obj.warp_and_save_slide(dst_f=dst_f, 
                                  crop="reference",                                  
                                  compression="jp2k", 
                                  Q=90,
                                  pyramid=False)

registration.kill_jvm()